<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinasDeLimpezaDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Pré - Processamento do texto

Classificação

Import e instalação das bibliotecas necessárias

In [17]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

In [18]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [19]:
# Definição de constantes

# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú0-9]+'

# Quantidade tokens tweets pequeno
QUANT_TOKENS_MIN = 5

########## PATHS DE ARQUIVOS ##########
# Base completa bruta
PATH_BASE_COMPLETA_BRUTA = r"/content/drive/MyDrive/TCC/dados/tweets_coletados/base80k.csv"

# Base teste bruta
PATH_BASE_TESTE_BRUTA = r"/content/drive/MyDrive/TCC/dados/tweets_coletados/base_teste.csv"

# Dicionário de palavrões
PATH_DICT_PALAVROES = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

# Dicionário de internetes
PATH_DICT_INTERNETES = r"/content/drive/MyDrive/TCC/dados/dicionarios/internetes_list.csv"

# Dicionário de estados
PATH_DICT_ESTADOS = r"/content/drive/MyDrive/TCC/dados/dicionarios/estados_list.csv"

# Dicionário de expressões contraídas
PATH_DICT_EXPRESSOES = r"/content/drive/MyDrive/TCC/dados/dicionarios/dict_hate_contraidos.csv"

# Lista de vogais duplicadas
PATH_LIST_VOGAIS_DUPLICADAS = r"/content/drive/MyDrive/TCC/dados/dicionarios/vogais_duplicadas.csv"

# Lista de stopWords
PATH_LIST_STOP_WORDS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/stop_word_list.txt"



In [20]:
#Carrega a base de dados
baseTweets = pd.read_csv(PATH_BASE_TESTE_BRUTA, index_col=0)
baseTweets.head()

,id_str,created_at,text,entities.hashtags,place.full_name
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil"
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil"
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil"
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil"
4,1148607640383217664,Tue Jul 09 14:59:36 +0000 2019,@conexaopolitica É um bundão ....,[],"Biguaçu, Brasil"


In [21]:
baseTweets.shape

(1000, 5)

Fase 1
Definição das funções usadas na limpeza Etapas:

1. Remover menções (Remover @fulano)
2. Remover caracteres especiais Remover caracteres de pontuação
3. Remover emojis
4. Expandir internetês
5. Normalizar nomes de estados (SP->São Paulo)
6. Retirar letras repetidas
7. Remover tweets com <5 tokens
8. Remover tweets repetidos (texto idêntico)

In [22]:
# Main
def pre_processamento(coluna_text):
    baseTweets['text_limpo'] = coluna_text # Inicializa nova coluna
    indice = 0
    tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
    dict_internetes = pd.read_csv(PATH_DICT_INTERNETES)
    dict_estados = pd.read_csv(PATH_DICT_ESTADOS)
    dict_expressoes = pd.read_csv(PATH_DICT_EXPRESSOES, encoding='latin-1')
    list_vogais_duplicadas = pd.read_csv(PATH_LIST_VOGAIS_DUPLICADAS)
    for text in coluna_text:
        text = remover_mencao(text.lower()) # Remover menções
        text = re.sub(r'http\S+', '', text) # Remover links
        tokens = tokenizer.tokenize(text) # Remover caracteres especiais, pontuação e emojis
        tokens = expandir_sigla(tokens,dict_internetes) # Expandir Internetes
        tokens = expandir_sigla(tokens,dict_estados) # Normalizar nomes de estado
        tokens = remover_letras_repetidas(tokens, list_vogais_duplicadas) # Retirar letras repetidas
        tokens = seta_tweet_pequeno_para_um(tokens) # Marca tweets pequenos

        text_limpo = " ".join(tokens)
        baseTweets['text_limpo'][indice] = text_limpo

        indice+=1 # Incrementa contador

    # Remover tweets marcados como pequenos e remove tweets repetidos(texto idêntico) 
    baseTweets.drop_duplicates(subset='text_limpo', keep=False, inplace=True)
    #baseTweets.reset_index(inplace=True)

    # apaga as linhas com o texto = '1'
    indexNames = baseTweets[ baseTweets['text_limpo'] == '1' ].index    
    baseTweets.drop(indexNames , inplace=True)


# Remover menções   
def remover_mencao(tweet):
    tweet_sem_mencao = re.sub(r'[@]\w+', '', tweet)
    tweet_sem_mencao = " ".join(tweet_sem_mencao.split()) # Remover espaços a mais
    return tweet_sem_mencao

# ESSA FUNÇÃO É CASE-SENSITIVE
def expandir_sigla(tokens, dicionario):
    dicionario_py = dicionario.to_dict() # Converte Dataframe para dicionário python
    keys = list(dicionario_py.keys()) # Dicionário de 2 colunas: chave, valor
    tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
    indice = 0
    for sigla in dicionario_py[keys[0]].values():
        if sigla in tokens:
            text = " ".join(tokens) # Agrupa os tokens formando o texto do tweet
            significado = dicionario_py[keys[1]][indice]
            text = re.sub(r'^' + sigla + '\s', significado + ' ',text) # Substitui no início
            text = re.sub(r'\s' + sigla + '$',' '+ significado,text) # Substitui no final
            text = re.sub(r'\s' + sigla + '\s', ' '+significado+ ' ',text) #Substitui no meio
            tokens = tokenizer.tokenize(text) # Tokeniza o texto do tweet após troca
            #print(tokens)

        indice+=1 # Incremento do contador
    return tokens

# Retirar letras repetidas
def remover_letras_repetidas(tokens, lista):
    lista = list(lista['vogais_duplicadas'])
    tokens_sem_letras_repetidas = []
    for token in tokens:
        flag = True
        if token not in lista: # Se for uma palavra com vogais dobradas não modifica o token
            for i in range(len(token)):
                # Se tiver "rr" ou "ss" não modifica o token
                try:
                    if (token[i].lower()=='r' and token[i+1].lower()=='r') or\
                        (token[i].lower()=='s' and token[i+1].lower()=='s'):
                        flag = False
                        break
                except:
                    break
            if flag:
                # Remove letras repetidas
                token = ''.join(c[0] for c in itertools.groupby(token))
        tokens_sem_letras_repetidas.append(token)
    return tokens_sem_letras_repetidas

# Marca tweets pequenos
def seta_tweet_pequeno_para_um(tokens):
    if len(tokens)<QUANT_TOKENS_MIN:
        tokens = '1'
    return tokens

In [23]:
# Aplicar pré processamento na base
pre_processamento(baseTweets['text'])
baseTweets.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito


In [24]:
# Porque está diminuindo quando rodamos mais de uma vez ?
baseTweets.shape

(773, 6)

In [25]:
baseTweets.to_csv('Teste_Base1_Com_StopWords.csv') #Salva base limpa

Fase 2
remover stopswords

In [26]:
# O arquivo "portuguese" contem Stopwords em Portugues
Sw = open(PATH_LIST_STOP_WORDS_PT_BR,'r',encoding='utf-8')
ptBr_stopwords = Sw.read()

In [27]:
baseTweets['text_cleanStopWords'] = baseTweets['text_limpo'].apply(lambda x: ' '.join([word for word in x.split() if word not in (ptBr_stopwords)]))
baseTweets.head()

,id_str,created_at,text,entities.hashtags,place.full_name,text_limpo,text_cleanStopWords
0,1037067688953167879,Tue Sep 04 19:59:39 +0000 2018,"O famoso: Deus me livre, mas quem me dera... h...",[],"Rio de Janeiro, Brazil",o famoso deus me livre mas quem me dera,famoso deus livre dera
1,1101497244325502977,Fri Mar 01 14:59:42 +0000 2019,Eu to ficando maluco ou o Weverton do Palmeira...,[],"Divinópolis, Brasil",eu estou ficando maluco ou o weverton do palme...,ficando maluco weverton palmeiras convocado se...
2,1085552210191233025,Wed Jan 16 14:59:50 +0000 2019,👉🏾 Hoje: SAMBINHA DE QUARTA \r\n📍Hashtag Bar -...,[],"Rio de Janeiro, Brazil",hoje sambinha de quarta hashtag bar seabra fil...,sambinha quarta hashtag bar seabra filho sexta...
3,1151869182373482501,Thu Jul 18 14:59:48 +0000 2019,@rick_pcf @ygorcac 2017 de primeira achei feio...,[],"Fortaleza, Brazil",2017 de primeira achei feio talvez por ser mei...,2017 achei feio talvez meio diferente demorei ...
5,1074695634513469440,Mon Dec 17 15:59:40 +0000 2018,P= mt chato mas eu gosto mt❤️,[],"Maricá, Brasil",p muito chato mas eu gosto muito,chato gosto


In [28]:
baseTweets.to_csv('Teste_Base2_Sem_StopWords.csv') #Salva base limpa